In [ ]:
# Notebook to duplicate ARGOG using cells 
## code is from main.py

In [ ]:
# !pip install --upgrade llama-index
# !pip show llama-index

# !pip install llama-index-agent-openai  llama-index-cli  llama-index-core  llama-index-embeddings-openai  llama-index-indices-managed-llama-cloud  llama-index-legacy  llama-index-llms-openai  llama-index-multi-modal-llms-openai  llama-index-program-openai  llama-index-question-gen-openai  llama-index-readers-file  llama-index-readers-llama-parse  nltk
# !pip install -U llama-index
# %pip install -q cohere llama-index-postprocessor-cohere-rerank
# !pip install tonic-validate

In [5]:
# importing stuff
import openai
import chromadb
from dotenv import load_dotenv
import pandas as pd
from utils import run_experiment, load_config
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, PromptTemplate, ServiceContext, Settings
# from llama_index.settings import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine, RetrieverQueryEngine
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.postprocessor import LLMRerank
from llama_index.core.retrievers import QueryFusionRetriever
from tonic_validate import ValidateScorer, ValidateApi
from tonic_validate.metrics import RetrievalPrecisionMetric, AnswerSimilarityMetric
import os

In [7]:
### SETUP --------------------------------------------------------------------------------------------------------------
# Load the config file (.env vars)
load_config()
load_dotenv(override=True)
# Set the OpenAI API key for authentication.
openai.api_key = os.getenv("OPENAI_API_KEY")
tonic_validate_api_key = os.getenv("HASSAN_TONIC_VALIDATE_API1")
tonic_validate_project_key = os.getenv("HASSAN_TONIC_VALIDATE_PROJECT1")
tonic_validate_benchmark_key = os.getenv("HASSAN_TONIC_VALIDATE_BENCHMARK1")
validate_api = ValidateApi(tonic_validate_api_key)
# print (f'{tonic_validate_api_key} {tonic_validate_project_key} {tonic_validate_benchmark_key}')


In [8]:
# Service context
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.0)
embed_model = OpenAIEmbedding(model="text-embedding-3-large")
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")
# service_context = ServiceContext.from_defaults(llm = llm, embed_model = embed_model)
# settings = Settings(llm=llm, embed_model=embed_model)
Settings.llm = llm 

chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [9]:
# Traditional VDB
chroma_collection = chroma_client.get_collection("ai_arxiv_full")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [10]:
# Sentence window VDB
chroma_collection_sentence_window = chroma_client.get_collection("ai_arxiv_sentence_window")
vector_store_sentence_window = ChromaVectorStore(chroma_collection=chroma_collection_sentence_window)
index_sentence_window = VectorStoreIndex.from_vector_store(vector_store=vector_store_sentence_window)


In [11]:
# Document summary VDB
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

In [12]:
# storage_context = StorageContext.from_defaults(persist_dir="Obelix")
storage_context = StorageContext.from_defaults(persist_dir="ai_arxiv_doc_summary")
doc_summary_index = load_index_from_storage(llm=llm,
                                              storage_context=storage_context,
                                             embed_model = embed_model)

In [13]:
# Prompt template
with open("resources/text_qa_template.txt", 'r', encoding='utf-8') as file:
    text_qa_template_str = file.read()

text_qa_template = PromptTemplate(text_qa_template_str)

In [14]:
# Tonic Validate setup
benchmark = validate_api.get_benchmark(tonic_validate_benchmark_key)
scorer = ValidateScorer(metrics=[RetrievalPrecisionMetric(), AnswerSimilarityMetric()],
                        model_evaluator="gpt-3.5-turbo")

### Define query engines -------------------------------------------------------------------------------------------------


In [15]:
# Naive RAG
query_engine_naive = index.as_query_engine(llm = llm,
                                           text_qa_template=text_qa_template,
                                           similarity_top_k=3,
                                           embed_model=embed_model)

In [ ]:
# Cohere Rerank
cohere_rerank = CohereRerank(api_key=config['cohere_api_key'], top_n=3)  # Ensure top_n matches k in naive RAG for comparability
query_engine_rerank = index.as_query_engine(
    similarity_top_k=10,
    text_qa_template=text_qa_template,
    node_postprocessors=[cohere_rerank],
    llm = llm,
    embed_model=embed_model
)